In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
#import keras

import statistics

from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_curve, auc
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.models import Model

In [2]:
def my_mode( input_data ):

    freq_count = dict() 

    for d in input_data: 
        if ( d in freq_count ):
            freq_count[d] += 1 
        else: 
            freq_count[d] = 1 

    max_count = 0 
    mode_value = None 

    for k,v in freq_count.items():
        if ( v > max_count ):
            mode_value = k 
            max_count = v

    return mode_value ; 

In [3]:
def normalise_data(input_data):

    #print(input_data[0:100])
    max_value = np.argmax(input_data)
    min_value = np.argmin(input_data)
    for i in range(0, len(input_data)):
        input_data[i] = (input_data[i] - min_value)/(max_value-min_value)
    return input_data

In [16]:
def generate_data_samples( input_dataset, x_columns, y_column, sample_size, y_compress_function=my_mode):

    y_values = [] 

    # First take the y series data and apply the compression function to get a single value per sample. 
    y_series = input_dataset[y_column]; 

    for i in range(0,len(input_dataset), sample_size): 

        if ( (i + 128) <= len(y_series)):
            value = y_compress_function(y_series[i:i+sample_size])
            y_values.append(value)
            

    # Now truncate the dataset, since we need the rows to be divisible by the sample size. 
    input_dataset = input_dataset.truncate(after=len(y_values*sample_size)-1)

    input_dataset = input_dataset[x_columns]
    #input_dataset.replace(np.nan, 0)
    np_x_values = input_dataset.to_numpy()
    np_x_values = np_x_values.astype(np.float32)
    np_x_values = np.nan_to_num(np_x_values, 0)
    
    #Regularize the data 
    #np_x_values.shape
#    for i in range (0, np_x_values.shape[1]): 
#        normalise_data(np_x_values[:,i])     
    np_x_values.shape
    np_x_values = np_x_values.reshape(-1, sample_size, np_x_values.shape[1] )
    np_x_values = np_x_values.reshape(np_x_values.shape[0], np_x_values.shape[1], np_x_values.shape[2], 1) 
    
    np_y_values = np.array(y_values); 
#   np_y_values = np_y_values.reshape(np_y_values.shape[0], 1)

    return np_x_values.astype(np.float32), np_y_values.astype(np.int32) 

In [5]:
def getFileList(rootFolder, searchExp = None):

    filesFound = [] 

    for root, subdirs, files in os.walk(rootFolder): 
        for fileName in files :
            if ( (searchExp is None) or re.search(searchExp, fileName) ):
                filesFound.append(os.path.join(root,fileName)) 
    return filesFound

In [6]:
import os
import re
file_names = getFileList('./Training', searchExp = "rawData.*l.csv")
print(file_names)

['./Training\\10_BOS_LeftLowerLimb\\rawData_Day1_l.csv', './Training\\10_BOS_LeftLowerLimb\\rawData_Day4_l.csv', './Training\\13_BOS_LeftLowerLimb\\rawData_Day1_l.csv', './Training\\13_BOS_LeftLowerLimb\\rawData_Day4_l.csv', './Training\\15_BOS_LeftLowerLimb\\rawData_Day1_l.csv', './Training\\15_BOS_LeftLowerLimb\\rawData_Day4_l.csv', './Training\\16_BOS_LeftLowerLimb\\rawData_Day1_l.csv', './Training\\16_BOS_LeftLowerLimb\\rawData_Day4_l.csv', './Training\\17_BOS_LeftLowerLimb\\rawData_Day1_l.csv', './Training\\17_BOS_LeftLowerLimb\\rawData_Day4_l.csv', './Training\\3_BOS_LeftLowerLimb\\rawData_Day1_l.csv', './Training\\3_BOS_LeftLowerLimb\\rawData_Day4_l.csv', './Training\\4_BOS_LeftLowerLimb\\rawData_Day1_l.csv', './Training\\4_BOS_LeftLowerLimb\\rawData_Day4_l.csv', './Training\\5_BOS_LeftLowerLimb\\rawData_Day1_l.csv', './Training\\5_BOS_LeftLowerLimb\\rawData_Day4_l.csv', './Training\\6_BOS_LeftLowerLimb\\rawData_Day1_l.csv', './Training\\6_BOS_LeftLowerLimb\\rawData_Day4_l.csv',

In [5]:
#file_names = ["./3_BOS_LeftLowerLimb/rawData_Day1_l.csv", "./3_BOS_LeftLowerLimb/rawData_Day4_l.csv",
             "./4_BOS_LeftLowerLimb/rawData_Day1_l.csv", "./4_BOS_LeftLowerLimb/rawData_Day4_l.csv",
             "./5_BOS_LeftLowerLimb/rawData_Day1_l.csv", "./5_BOS_LeftLowerLimb/rawData_Day4_l.csv", 
             "./6_BOS_LeftLowerLimb/rawData_Day1_l.csv", "./6_BOS_LeftLowerLimb/rawData_Day4_l.csv",
             "./7_BOS_LeftLowerLimb/rawData_Day1_l.csv", "./7_BOS_LeftLowerLimb/rawData_Day4_l.csv",
             "./8_BOS_LeftLowerLimb/rawData_Day1_l.csv", "./8_BOS_LeftLowerLimb/rawData_Day4_l.csv", 
             "./9_BOS_LeftLowerLimb/rawData_Day1_l.csv", "./9_BOS_LeftLowerLimb/rawData_Day4_l.csv",
             "./10_BOS_LeftLowerLimb/rawData_Day1_l.csv", "./10_BOS_LeftLowerLimb/rawData_Day4_l.csv",
             "./13_BOS_LeftLowerLimb/rawData_Day1_l.csv", "./13_BOS_LeftLowerLimb/rawData_Day4_l.csv",
#             "./14_BOS_LeftLowerLimb/rawData_Day1_l.csv", "./14_BOS_LeftLowerLimb/rawData_Day4_l.csv",
             "./15_BOS_LeftLowerLimb/rawData_Day1_l.csv", "./15_BOS_LeftLowerLimb/rawData_Day4_l.csv",
             "./16_BOS_LeftLowerLimb/rawData_Day1_l.csv", "./16_BOS_LeftLowerLimb/rawData_Day4_l.csv",
             "./17_BOS_LeftLowerLimb/rawData_Day1_l.csv", "./17_BOS_LeftLowerLimb/rawData_Day4_l.csv"]

Create a list of file names for each patient, so that we can take data for days 1 and 4 and run them in sequence.

Merge data for Patients BOS_12 and BOS_19, for days 1 and 4, to create a single dataframe that will be used for validation.

In [25]:
valdata_day_12_1 = pd.read_csv("./12_BOS_LeftLowerLimb/rawData_Day1_l.csv")
valdata_day_12_4 = pd.read_csv("./12_BOS_LeftLowerLimb/rawData_Day4_l.csv")
valdata_12 = pd.concat([valdata_day_12_1, valdata_day_12_4], ignore_index=True)
valdata_day_19_1 = pd.read_csv("./19_BOS_LeftLowerLimb/rawData_Day1_l.csv")
valdata_day_19_4 = pd.read_csv("./19_BOS_LeftLowerLimb/rawData_Day4_l.csv")
valdata_19 = pd.concat([valdata_day_19_1, valdata_day_19_4], ignore_index=True)
valdata_19.head()
valdata = pd.concat([valdata_12, valdata_19], ignore_index=True)
X_val, Y_val = generate_data_samples(valdata, ['leftAnkle_X', 'leftAnkle_Y','leftAnkle_Z'], 'Dysk_Score', 128)
Y_val = tf.compat.v1.one_hot(Y_val, depth=5)
#print(X)
#print(Y)
print(len(valdata))
#print(X_val[0:10])

10203483


Merge data for Patients BOS_11 and BOS_18, for days 1 and 4, to create a single dataframe that will be used for testing.

In [28]:
testdata_day_11_1 = pd.read_csv("./11_BOS_LeftLowerLimb/rawData_Day1_l.csv")
testdata_day_11_4 = pd.read_csv("./11_BOS_LeftLowerLimb/rawData_Day4_l.csv")
testdata_11 = pd.concat([testdata_day_11_1, testdata_day_11_4], ignore_index=True)
testdata_day_18_1 = pd.read_csv("./18_BOS_LeftLowerLimb/rawData_Day1_l.csv")
testdata_day_18_4 = pd.read_csv("./18_BOS_LeftLowerLimb/rawData_Day4_l.csv")
testdata_18 = pd.concat([testdata_day_18_1, testdata_day_18_4], ignore_index=True)
testdata_18.head()
testdata = pd.concat([testdata_11, testdata_18], ignore_index=True)
X_test, Y_test = generate_data_samples(testdata, ['leftAnkle_X', 'leftAnkle_Y','leftAnkle_Z'], 'Dysk_Score', 128)
Y_test = tf.compat.v1.one_hot(Y_test, depth=5)
#print(X)
#print(Y)
print(len(testdata))
#print(X_test[0:10])

9957286


Set up hyperparameters

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Flatten, Conv1D, Conv2D, MaxPooling1D, MaxPooling2D
from tensorflow.keras.optimizers import Adam, schedules, SGD

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(# input_shape=(batch, 128, 3)
        8, (4, 3),
        padding="same",
        activation="relu",
        input_shape=(128, 3, 1)), #output_shape=(batch, 128, 3, 8)
    tf.keras.layers.MaxPool2D((3, 3)),  # (batch, 42, 1, 8)
    tf.keras.layers.Dropout(0.1),  # (batch, 42, 1, 8)
    tf.keras.layers.Conv2D(16, (4, 1), padding="same", activation="relu", input_shape=(42, 1, 16)),
    tf.keras.layers.MaxPool2D((3, 1), padding="same"),  # (batch, 14, 1, 16)
    tf.keras.layers.Dropout(0.1),  # (batch, 14, 1, 16)
    tf.keras.layers.Flatten(),  # (batch, 224)
    tf.keras.layers.Dense(16, activation="relu"),  # (batch, 16)
    tf.keras.layers.Dropout(0.1),  # (batch, 16)
    tf.keras.layers.Dense(5, activation="softmax")  # (batch, 4)
])

In [18]:
METRICS = [
    tf.keras.metrics.TruePositives(name='tp'),
    tf.keras.metrics.FalsePositives(name='fp'),
    tf.keras.metrics.TrueNegatives(name='tn'),
    tf.keras.metrics.FalseNegatives(name='fn'),
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall'),
    tf.keras.metrics.CategoricalAccuracy(name='acc'),
    tf.keras.metrics.AUC(name='auc'),
]

In [ ]:
model.summary()

The model is a classifier with multiple classes. Thus, the fitting includes class weights. In most cases, classes are not perfectly balanced, so this addition takes this into account.

In [19]:
# compile the model
model.compile(optimizer="adam",
              loss='categorical_crossentropy',
              metrics=[METRICS])
#model.compile(optimizer = tf.keras.optimizers.Adam(),
#              loss = 'sparse_categorical_crossentropy',
#              metrics=['accuracy'])

In [21]:
# fit - train the model
#class_weight = {0: 1., 1: 48., 2: 137., 3: 543., 4: 7780.,}
for data_file in file_names:
    training_dataframe = pd.read_csv(data_file)
    print(data_file)
    X_train, Y_train = generate_data_samples(training_dataframe, ['leftAnkle_X', 'leftAnkle_Y','leftAnkle_Z'], 'Dysk_Score', 128)
#hot encode the labels for 5 classes
    Y_train = tf.compat.v1.one_hot(Y_train, depth=5)
    print(len(X_train))
    print(len(Y_train))
    model.fit(x=X_train, y=Y_train, epochs=10) #class_weight=class_weight)
        #verbose=1, 
        #batch_size=128,
        #validation_data=(X_val, Y_val)

./Training\10_BOS_LeftLowerLimb\rawData_Day1_l.csv
19874
19874
Epoch 1/10
622/622 [==============================] - 3s 5ms/step - loss: 0.0390 - tp: 19799.0000 - fp: 75.0000 - tn: 79421.0000 - fn: 75.0000 - precision: 0.9962 - recall: 0.9962 - acc: 0.9962 - auc: 0.9990
Epoch 2/10
622/622 [==============================] - 3s 5ms/step - loss: 0.0206 - tp: 19799.0000 - fp: 75.0000 - tn: 79421.0000 - fn: 75.0000 - precision: 0.9962 - recall: 0.9962 - acc: 0.9962 - auc: 0.9995
Epoch 3/10
622/622 [==============================] - 3s 5ms/step - loss: 0.0189 - tp: 19800.0000 - fp: 74.0000 - tn: 79422.0000 - fn: 74.0000 - precision: 0.9963 - recall: 0.9963 - acc: 0.9963 - auc: 0.9994
Epoch 4/10
622/622 [==============================] - 3s 5ms/step - loss: 0.0191 - tp: 19800.0000 - fp: 74.0000 - tn: 79422.0000 - fn: 74.0000 - precision: 0.9963 - recall: 0.9963 - acc: 0.9963 - auc: 0.9996
Epoch 5/10
622/622 [==============================] - 3s 4ms/step - loss: 0.0177 - tp: 19800.0000 - fp: 7

599/599 [==============================] - 3s 5ms/step - loss: 0.0551 - tp: 18974.0000 - fp: 180.0000 - tn: 76436.0000 - fn: 180.0000 - precision: 0.9906 - recall: 0.9906 - acc: 0.9906 - auc: 0.9976
Epoch 9/10
599/599 [==============================] - 3s 5ms/step - loss: 0.0527 - tp: 18974.0000 - fp: 180.0000 - tn: 76436.0000 - fn: 180.0000 - precision: 0.9906 - recall: 0.9906 - acc: 0.9906 - auc: 0.9977
Epoch 10/10
599/599 [==============================] - 3s 5ms/step - loss: 0.0605 - tp: 18974.0000 - fp: 180.0000 - tn: 76436.0000 - fn: 180.0000 - precision: 0.9906 - recall: 0.9906 - acc: 0.9906 - auc: 0.9978
./Training\15_BOS_LeftLowerLimb\rawData_Day1_l.csv
18742
18742
Epoch 1/10
586/586 [==============================] - 3s 5ms/step - loss: 0.0531 - tp: 18638.0000 - fp: 104.0000 - tn: 74864.0000 - fn: 104.0000 - precision: 0.9945 - recall: 0.9945 - acc: 0.9945 - auc: 0.9982
Epoch 2/10
586/586 [==============================] - 3s 5ms/step - loss: 0.0411 - tp: 18638.0000 - fp: 104

Epoch 5/10
609/609 [==============================] - 3s 5ms/step - loss: 0.0681 - tp: 19197.0000 - fp: 273.0000 - tn: 77607.0000 - fn: 273.0000 - precision: 0.9860 - recall: 0.9860 - acc: 0.9860 - auc: 0.9984
Epoch 6/10
609/609 [==============================] - 3s 5ms/step - loss: 0.0666 - tp: 19197.0000 - fp: 273.0000 - tn: 77607.0000 - fn: 273.0000 - precision: 0.9860 - recall: 0.9860 - acc: 0.9860 - auc: 0.9985
Epoch 7/10
609/609 [==============================] - 3s 5ms/step - loss: 0.0675 - tp: 19197.0000 - fp: 273.0000 - tn: 77607.0000 - fn: 273.0000 - precision: 0.9860 - recall: 0.9860 - acc: 0.9860 - auc: 0.9986
Epoch 8/10
609/609 [==============================] - 3s 5ms/step - loss: 0.0664 - tp: 19197.0000 - fp: 273.0000 - tn: 77607.0000 - fn: 273.0000 - precision: 0.9860 - recall: 0.9860 - acc: 0.9860 - auc: 0.9986
Epoch 9/10
609/609 [==============================] - 3s 5ms/step - loss: 0.0654 - tp: 19197.0000 - fp: 273.0000 - tn: 77607.0000 - fn: 273.0000 - precision: 0.

601/601 [==============================] - 3s 5ms/step - loss: 0.0862 - tp: 18802.0000 - fp: 414.0000 - tn: 76450.0000 - fn: 414.0000 - precision: 0.9785 - recall: 0.9785 - acc: 0.9785 - auc: 0.9982
Epoch 3/10
601/601 [==============================] - 3s 5ms/step - loss: 0.0851 - tp: 18802.0000 - fp: 414.0000 - tn: 76450.0000 - fn: 414.0000 - precision: 0.9785 - recall: 0.9785 - acc: 0.9785 - auc: 0.9982
Epoch 4/10
601/601 [==============================] - 3s 5ms/step - loss: 0.0840 - tp: 18802.0000 - fp: 414.0000 - tn: 76450.0000 - fn: 414.0000 - precision: 0.9785 - recall: 0.9785 - acc: 0.9785 - auc: 0.9981
Epoch 5/10
601/601 [==============================] - 3s 5ms/step - loss: 0.0843 - tp: 18802.0000 - fp: 414.0000 - tn: 76450.0000 - fn: 414.0000 - precision: 0.9785 - recall: 0.9785 - acc: 0.9785 - auc: 0.9982
Epoch 6/10
601/601 [==============================] - 3s 5ms/step - loss: 0.0823 - tp: 18802.0000 - fp: 414.0000 - tn: 76450.0000 - fn: 414.0000 - precision: 0.9785 - reca

598/598 [==============================] - 3s 5ms/step - loss: 0.0741 - tp: 18773.0000 - fp: 353.0000 - tn: 76151.0000 - fn: 353.0000 - precision: 0.9815 - recall: 0.9815 - acc: 0.9815 - auc: 0.9982
./Training\5_BOS_LeftLowerLimb\rawData_Day4_l.csv
21362
21362
Epoch 1/10
668/668 [==============================] - 3s 5ms/step - loss: 0.0361 - tp: 21158.0000 - fp: 204.0000 - tn: 85244.0000 - fn: 204.0000 - precision: 0.9905 - recall: 0.9905 - acc: 0.9905 - auc: 0.9994
Epoch 2/10
668/668 [==============================] - 3s 5ms/step - loss: 0.0324 - tp: 21158.0000 - fp: 204.0000 - tn: 85244.0000 - fn: 204.0000 - precision: 0.9905 - recall: 0.9905 - acc: 0.9905 - auc: 0.9995
Epoch 3/10
668/668 [==============================] - 3s 5ms/step - loss: 0.0314 - tp: 21158.0000 - fp: 204.0000 - tn: 85244.0000 - fn: 204.0000 - precision: 0.9905 - recall: 0.9905 - acc: 0.9905 - auc: 0.9995
Epoch 4/10
668/668 [==============================] - 3s 5ms/step - loss: 0.0312 - tp: 21158.0000 - fp: 204.0

669/669 [==============================] - 4s 5ms/step - loss: 0.0214 - tp: 21329.0000 - fp: 77.0000 - tn: 85547.0000 - fn: 77.0000 - precision: 0.9964 - recall: 0.9964 - acc: 0.9964 - auc: 0.9989
Epoch 8/10
669/669 [==============================] - 3s 5ms/step - loss: 0.0229 - tp: 21329.0000 - fp: 77.0000 - tn: 85547.0000 - fn: 77.0000 - precision: 0.9964 - recall: 0.9964 - acc: 0.9964 - auc: 0.9989
Epoch 9/10
669/669 [==============================] - 3s 5ms/step - loss: 0.0225 - tp: 21329.0000 - fp: 77.0000 - tn: 85547.0000 - fn: 77.0000 - precision: 0.9964 - recall: 0.9964 - acc: 0.9964 - auc: 0.9990
Epoch 10/10
669/669 [==============================] - 3s 5ms/step - loss: 0.0441 - tp: 21328.0000 - fp: 78.0000 - tn: 85546.0000 - fn: 78.0000 - precision: 0.9964 - recall: 0.9964 - acc: 0.9964 - auc: 0.9988
./Training\7_BOS_LeftLowerLimb\rawData_Day4_l.csv
16585
16585
Epoch 1/10
519/519 [==============================] - 2s 5ms/step - loss: 0.1203 - tp: 16236.0000 - fp: 349.0000 - t

661/661 [==============================] - 3s 5ms/step - loss: 0.0877 - tp: 20727.0000 - fp: 401.0000 - tn: 84111.0000 - fn: 401.0000 - precision: 0.9810 - recall: 0.9810 - acc: 0.9810 - auc: 0.9978
Epoch 5/10
661/661 [==============================] - 3s 5ms/step - loss: 0.1001 - tp: 20727.0000 - fp: 401.0000 - tn: 84111.0000 - fn: 401.0000 - precision: 0.9810 - recall: 0.9810 - acc: 0.9810 - auc: 0.9977
Epoch 6/10
661/661 [==============================] - 3s 5ms/step - loss: 0.0885 - tp: 20727.0000 - fp: 401.0000 - tn: 84111.0000 - fn: 401.0000 - precision: 0.9810 - recall: 0.9810 - acc: 0.9810 - auc: 0.9977
Epoch 7/10
661/661 [==============================] - 3s 5ms/step - loss: 0.0849 - tp: 20727.0000 - fp: 401.0000 - tn: 84111.0000 - fn: 401.0000 - precision: 0.9810 - recall: 0.9810 - acc: 0.9810 - auc: 0.9980
Epoch 8/10
661/661 [==============================] - 3s 5ms/step - loss: 0.0836 - tp: 20727.0000 - fp: 401.0000 - tn: 84111.0000 - fn: 401.0000 - precision: 0.9810 - reca

In [33]:
#evaluate val_dataset before testing on test data. 
model.evaluate(X_val, Y_val)
classifications = model.predict(X_val)
print(classifications[0])
print(Y_val[0])
pred = np.argmax(model.predict(X_val), axis=1)
confusion = tf.math.confusion_matrix(labels=tf.keras.backend.argmax(Y_val,axis = 1), #labels=tf.constant(Y_test),
                                       predictions=tf.constant(pred).numpy(), num_classes=5)
print(confusion)

2492/2492 [==============================] - 8s 3ms/step - loss: 1.5336 - tp: 78662.0000 - fp: 1052.0000 - tn: 317804.0000 - fn: 1052.0000 - precision: 0.9868 - recall: 0.9868 - acc: 0.9868 - auc: 0.9934
[1.0000000e+00 1.1978645e-18 0.0000000e+00 0.0000000e+00 0.0000000e+00]
tf.Tensor([1. 0. 0. 0. 0.], shape=(5,), dtype=float32)
tf.Tensor(
[[78662     0     0     0     0]
 [  742     0     0     0     0]
 [  260     0     0     0     0]
 [   50     0     0     0     0]
 [    0     0     0     0     0]], shape=(5, 5), dtype=int32)


In [32]:
model.evaluate(X_test, Y_test)
pred = np.argmax(model.predict(X_test), axis=1)
#confusion = tf.math.confusion_matrix(labels=tf.keras.backend.argmax(Y_test,axis = 1),
#                                       predictions=tf.keras.backend.argmax(pred, axis = 1), num_classes=5)

confusion = tf.math.confusion_matrix(labels=tf.keras.backend.argmax(Y_test,axis = 1), #labels=tf.constant(Y_test),
                                       predictions=tf.constant(pred).numpy(), num_classes=5)
print(confusion)

#print("Loss {}, Accuracy {}".format(loss, acc))
# Convert the model to the TensorFlow Lite format without quantization
#converter = tf.lite.TFLiteConverter.from_keras_model(model)
#tflite_model = converter.convert()

2431/2431 [==============================] - 7s 3ms/step - loss: 6.8972 - tp: 75460.0000 - fp: 2331.0000 - tn: 308833.0000 - fn: 2331.0000 - precision: 0.9700 - recall: 0.9700 - acc: 0.9700 - auc: 0.9853
tf.Tensor(
[[75460     0     0     0     0]
 [ 1613     0     0     0     0]
 [  565     0     0     0     0]
 [  143     0     0     0     0]
 [   10     0     0     0     0]], shape=(5, 5), dtype=int32)


In [31]:
from tensorflow.python.framework import ops
from tensorflow.python.ops.numpy_ops import np_dtypes
from tensorflow.python.ops.numpy_ops import np_config
from tensorflow.python.ops.numpy_ops import np_export
from tensorflow.python.ops.numpy_ops import np_math_ops
np_config.enable_numpy_behavior()

con_mat_norm = np.around(confusion.astype('float') / confusion.sum(axis=1)[:, np.newaxis], decimals=2)
con_mat_df = pd.DataFrame(con_mat_norm,
                     index = classes, 
                     columns = classes)

AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'sum'

In [34]:
!mkdir -p saved_model
model.save('saved_model/my_CNNmodel')

A subdirectory or file -p already exists.
Error occurred while processing: -p.
A subdirectory or file saved_model already exists.
Error occurred while processing: saved_model.


INFO:tensorflow:Assets written to: saved_model/my_CNNmodel\assets


In [35]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
#converter = tf.lite.TFLiteConverter.from_keras_model('CNNmodel.h5')
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: C:\Users\svetl\AppData\Local\Temp\tmp81ih2fic\assets


In [38]:
import pathlib
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# These options are for converter optimizaitons
# Try the converter without them and explore model size and accuracy
# Then use them and reconvert the model and explore model
# size an accuracy at that point.

converter.optimizations = [tf.lite.Optimize.DEFAULT]    # Uncomment this line for Model 2 and Model 3

def representative_data_gen():                          # Uncomment the following 5 lines for Model 3
     for input_value, _ in X_test.take(100):
         yield [input_value]
converter.representative_dataset = representative_data_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]

tflite_model = converter.convert()
#tflite_models_file = pathlib.Path("./tmp/CNNmodel1.tflite")
tflite_model_file = pathlib.Path("./tmp/CNNmodel2.tflite")     # Change the filename here for Model2 and Model3!
tflite_model_file.write_bytes(tflite_model)
# Without any optimizations I got
# 20560  (model1.tflite) - the model that was created with 10 epochs
# With the .optimizations property set I got
# 9728 (model2.tflite)


INFO:tensorflow:Assets written to: C:\Users\svetl\AppData\Local\Temp\tmplyd1x0a4\assets


INFO:tensorflow:Assets written to: C:\Users\svetl\AppData\Local\Temp\tmplyd1x0a4\assets


TypeError: 'numpy.float32' object is not iterable

In [29]:
#Run this cell each time to test your model's accuracy (make sure to change the filename). 
#Tqdm is a Python library used to display smart progress bars that show the progress of Python code execution.
from tqdm import tqdm
# Load TFLite model and allocate tensors.
tflite_model_file = './tmp/model1.tflite'                 # Change the filename here for Model 2 and 3 with optimisations
interpreter = tf.lite.Interpreter(model_path=tflite_model_file)
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]
print(input_index)
print(output_index)

predictions = []

for i in tqdm(range(0,100)):
    data = X_test[i]
    data = data.reshape(1, data.shape[0], data.shape[1], data.shape[2]) 
    #print(data.shape)
    label = Y_test[i]
    interpreter.set_tensor(input_index, data)
    interpreter.invoke()
    predictions.append(interpreter.get_tensor(output_index))
    
#    Y_test_labels.append(label.numpy()[0])
#    X_test.append(X_test)

# For model 1, 
# For model 2, 
# For model 3, 
# Note: since the it/s will depend on the computer on which the model instance is running -- it would vary a bit.

score = 0
for item in range(0,100):
  prediction=np.argmax(predictions[item])
  label = Y_test[item]
  if prediction==label:
    score=score+1

print("Out of 100 predictions I got " + str(score) + " correct")

# Model 1 - 100 Correct
# Model 2 - 99 Correct
# Model 3 - 99 Correct
# Note: training starts from a random intialization the result could be off by 1 or 2 correct.

0
16


100%|██████████| 100/100 [00:00<00:00, 6252.32it/s]


Out of 100 predictions I got 100 correct
